In [7]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

# Input and output paths
input_dir = r"C:\Users\91976\Desktop\Hochshule Harz\Semester 3\Project work\input_dir"
output_dir = r"C:\Users\91976\Desktop\Hochshule Harz\Semester 3\Project work\Output_dir"
timelapse_dir = os.path.join(output_dir, "Timelapse_Video")
enhanced_dir = os.path.join(output_dir, "Enhanced_Images")
good_dir = os.path.join(output_dir, "Good_Images")
comparison_dir = os.path.join(output_dir, "Comparison")
all_images_dir = os.path.join(output_dir, "All_Images")

os.makedirs(enhanced_dir, exist_ok=True)
os.makedirs(good_dir, exist_ok=True)
os.makedirs(comparison_dir, exist_ok=True)
os.makedirs(all_images_dir, exist_ok=True)
os.makedirs(timelapse_dir, exist_ok=True)

# Function to detect if an image is a daylight image
def is_daylight_image(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness = np.mean(hsv[:, :, 2])
    return brightness > 120  # Adjusted threshold for better daylight detection

# Advanced image enhancement for fog and rain removal
def enhance_image(image):
    # Convert to YUV for better contrast enhancement
    yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    yuv[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])
    enhanced_img = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    
    # Apply dehazing using contrast-limited adaptive histogram equalization
    lab = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    enhanced_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    
    # Apply bilateral filter for noise reduction
    enhanced_img = cv2.bilateralFilter(enhanced_img, 9, 75, 75)
    
    return enhanced_img

# Process images
def process_images():
    enhanced_images = []
    comparisons = []

    for file_name in os.listdir(input_dir):
        file_path = os.path.join(input_dir, file_name)
        
        if not file_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        
        original_img = cv2.imread(file_path)
        if not is_daylight_image(original_img):
            continue  # Skip non-daylight images
        
        enhanced_img = enhance_image(original_img)
        
        enhanced_file_path = os.path.join(enhanced_dir, file_name)
        cv2.imwrite(enhanced_file_path, enhanced_img)
        
        # Save comparison image
        comparison_img = np.hstack((original_img, enhanced_img))
        comparison_file_path = os.path.join(comparison_dir, file_name)
        cv2.imwrite(comparison_file_path, comparison_img)
        comparisons.append(file_name)
        
        # Copy to all images folder
        cv2.imwrite(os.path.join(all_images_dir, file_name), enhanced_img)
    
    return comparisons

# Create smooth timelapse video
def create_timelapse(image_folder, output_video):
    images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg") or img.endswith(".png")])
    
    if not images:
        print("No daylight images found for timelapse.")
        return
    
    first_image = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = first_image.shape
    
    video = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), 15, (width, height))
    
    for image in images:
        img_path = os.path.join(image_folder, image)
        frame = cv2.imread(img_path)
        video.write(frame)
    
    video.release()

# Main script
if __name__ == "__main__":
    comparisons = process_images()
    
    # Generate smooth timelapse video
    timelapse_video_path = os.path.join(timelapse_dir, "timelapse_daylight.mp4")
    create_timelapse(enhanced_dir, timelapse_video_path)
    
    print(f"Comparison images saved: {len(comparisons)}")
    print(f"Timelapse video saved at: {timelapse_video_path}")


Comparison images saved: 1058
Timelapse video saved at: C:\Users\91976\Desktop\Hochshule Harz\Semester 3\Project work\Output_dir\Timelapse_Video\timelapse_daylight.mp4
